<a href="https://colab.research.google.com/github/canterville184/FraudDetection/blob/main/FraudDetection_training_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -U 'hopsworks[python]' --quiet

In [1]:
import pandas as pd
import os
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

# Creating Feature Groups in Feature Store

In [19]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213626


In [21]:
# Get or create the 'transactions' feature group
trans_fg = fs.get_or_create_feature_group(
    name="transactions",
    version=1,
    description="Transaction data",
    primary_key=["transaction_id"],
    event_time="date",
    online_enabled=True,
)

In [24]:
# Select features for training data
selected_features = trans_fg.select(['step', 'type', 'amount', 'nameorig', 'namedest', 'isfraud'])

In [27]:
selected_features.show(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (34.79s) 


,step,type,amount,nameorig,namedest,isfraud
0,95,CASH_IN,327060.99,C277542917,C18058083,0
1,19,CASH_OUT,98056.68,C1739707237,C1965963623,0
2,167,PAYMENT,3457.36,C1367949555,M194892973,0
3,18,CASH_IN,113256.95,C826616230,C1010549417,0
4,19,CASH_OUT,260417.16,C253927786,C2008807592,0
5,158,PAYMENT,20430.38,C606779003,M1972077676,0
6,8,CASH_OUT,235735.99,C1652837341,C1302931951,0
7,204,CASH_OUT,152630.81,C1173811052,C800039670,0
8,156,CASH_OUT,382510.91,C181000558,C1518436463,0
9,205,CASH_IN,93158.29,C1661116394,C100048256,0


In [28]:
# Import transformation functions from Hopsworks.
from hopsworks.hsfs.builtin_transformations import label_encoder

# Map features to transformations.
transformation_functions = [
    label_encoder("type"),
]

# Feature View Creation

In [29]:
# Get or create the 'transactions_view' feature view
feature_view = fs.get_or_create_feature_view(
    name='transactions_view',
    version=1,
    query=selected_features,
    labels=["type"],
    transformation_functions=transformation_functions,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213626/fs/1200267/fv/transactions_view/version/1
